In [45]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold ,StratifiedKFold, cross_val_score, cross_validate, GridSearchCV
import numpy as np
import pandas as pd

In [5]:
# 학습/테스트 데이터 셋 분리하지 않고 예측

iris_data = load_iris()
dt_clf = DecisionTreeClassifier()


train_data = iris_data.data
train_label = iris_data.target

#학습 수행
dt_clf.fit(train_data, train_label)

#테스트
pred = dt_clf.predict(train_data)
print(accuracy_score(train_label, pred))

1.0


* 예측을 train_data로 했기 때문에 결과 1.0으로 출력(잘못됨)
* 예측은 테스트 데이터로 해야함

In [7]:
# 학습/테스트 데이터셋 분리하고 예측
X_train, X_test, y_train, y_test= train_test_split(iris_data.data, iris_data.target, test_size=0.3, random_state=20)

y_train

array([0, 0, 0, 2, 0, 2, 2, 0, 2, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 0, 2, 0, 0, 0, 2, 1, 1, 2, 2, 1, 0, 2, 0, 1, 2, 0, 1, 1, 2, 0,
       0, 2, 2, 1, 1, 1, 2, 2, 0, 2, 0, 1, 0, 0, 2, 2, 2, 0, 0, 2, 1, 1,
       2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 1, 0, 1, 2, 0, 1])

In [8]:
#학습 수행
dt_clf.fit(X_train, y_train)

#예측 수행
pred = dt_clf.predict(X_test)
print(accuracy_score(y_test, pred))

0.8888888888888888


In [11]:
#판다스를 이용해 학습/테스트 데이터셋 분류

iris_df = pd.DataFrame(iris_data.data, columns = iris_data.feature_names)
iris_df['target'] = iris_data.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [13]:
#feature 데이터프레임 반환(마지막 열 제외)
feature_df = iris_df.iloc[:, :-1]

#target 데이터 프레임 반환
target_df = iris_df.iloc[:, -1]

#학습/테스트 데이터 분할
X_train, X_test, y_train, y_test= train_test_split(feature_df, target_df, test_size=0.3, random_state=4)

In [14]:
type(X_train)

pandas.core.frame.DataFrame

In [15]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print(accuracy_score(y_test, pred))

0.9777777777777777


# Data Split과 모델 검증
* 언제
    * 충분히 큰 데이터 세트를 가용할 때
    * 충분히 큰 데이터가 없을때에는 교차확인 고려
* 왜
    * 학습에 사용하지 않는

# K폴드 예제

In [16]:
iris = load_iris()
features = iris.data
label = iris.target

features.shape

(150, 4)

In [18]:
dt_clf = DecisionTreeClassifier(random_state=156)

#5개의 폴드 세트로 분리하는 KFold 객체 생성
kfold = KFold(n_splits=5)

#폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuracy = []

In [21]:
#폴드 별 학습용, 검증용 데이터 세트의 행 인덱스확인
for train_index, test_index in kfold.split(features):
    print(train_index, test_index)

[ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 1

In [22]:
for train_index, test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
        
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    acc = np.round(accuracy_score(y_test, pred), 3)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('학습 데이터 크기 : %d, 검증 데이터 크기 : %d, 정확도 : %f' % (train_size, test_size, acc))
    cv_accuracy.append(acc)
    
print('평균 정확도 : ', np.mean(cv_accuracy))

학습 데이터 크기 : 120, 검증 데이터 크기 : 30, 정확도 : 1.000000
학습 데이터 크기 : 120, 검증 데이터 크기 : 30, 정확도 : 0.967000
학습 데이터 크기 : 120, 검증 데이터 크기 : 30, 정확도 : 0.867000
학습 데이터 크기 : 120, 검증 데이터 크기 : 30, 정확도 : 0.933000
학습 데이터 크기 : 120, 검증 데이터 크기 : 30, 정확도 : 0.733000
평균 정확도 :  0.9


# KFold 문제점

In [26]:
iris = load_iris()

iris_df = pd.DataFrame(iris_data.data, columns = iris_data.feature_names)
iris_df['label'] = iris_data.target
iris_df.head()

In [29]:
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [32]:
#3개의 폴드 세트로 분리하는 KFold 객체 생성
kfold = KFold(n_splits=3)

n=0
for train_index, test_index in kfold.split(features):
    n =+ 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('교차검증 : %d' % n)
    print('학습용 : \n', label_train.value_counts())
    print('검증용 : \n', label_test.value_counts())

교차검증 : 1
학습용 : 
 1    50
2    50
Name: label, dtype: int64
검증용 : 
 0    50
Name: label, dtype: int64
교차검증 : 1
학습용 : 
 0    50
2    50
Name: label, dtype: int64
검증용 : 
 1    50
Name: label, dtype: int64
교차검증 : 1
학습용 : 
 0    50
1    50
Name: label, dtype: int64
검증용 : 
 2    50
Name: label, dtype: int64


In [34]:
dt_clf = DecisionTreeClassifier(random_state=156)

#3개의 폴드 세트로 분리하는 KFold 객체 생성
kfold = KFold(n_splits=3)

#폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuracy = []

for train_index, test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
        
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    #반복시마다 정확도 측정
    acc = np.round(accuracy_score(y_test, pred), 3)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('학습 데이터 크기 : %d, 검증 데이터 크기 : %d, 정확도 : %f' % (train_size, test_size, acc))
    cv_accuracy.append(acc)
    
print('평균 정확도 : ', np.mean(cv_accuracy))

학습 데이터 크기 : 100, 검증 데이터 크기 : 50, 정확도 : 0.000000
학습 데이터 크기 : 100, 검증 데이터 크기 : 50, 정확도 : 0.000000
학습 데이터 크기 : 100, 검증 데이터 크기 : 50, 정확도 : 0.000000
평균 정확도 :  0.0


# Stratified KFold

In [36]:
skf = StratifiedKFold(n_splits=3)
n=0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n += 1
    
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('교차검증 : %d' % n)
    print('학습용 레이블 분포 : \n', label_train.value_counts())
    print('검증용 레이블 분포 : \n', label_test.value_counts())

교차검증 : 1
학습용 레이블 분포 : 
 2    34
0    33
1    33
Name: label, dtype: int64
검증용 레이블 분포 : 
 0    17
1    17
2    16
Name: label, dtype: int64
교차검증 : 2
학습용 레이블 분포 : 
 1    34
0    33
2    33
Name: label, dtype: int64
검증용 레이블 분포 : 
 0    17
2    17
1    16
Name: label, dtype: int64
교차검증 : 3
학습용 레이블 분포 : 
 0    34
1    33
2    33
Name: label, dtype: int64
검증용 레이블 분포 : 
 1    17
2    17
0    16
Name: label, dtype: int64


In [37]:
dt_clf = DecisionTreeClassifier(random_state=156)

#3개의 폴드 세트로 분리하는 KFold 객체 생성
skfold = StratifiedKFold(n_splits=3)

#폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuracy = []

for train_index, test_index in skfold.split(features, label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
        
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    #반복시마다 정확도 측정
    acc = np.round(accuracy_score(y_test, pred), 3)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('학습 데이터 크기 : %d, 검증 데이터 크기 : %d, 정확도 : %f' % (train_size, test_size, acc))
    cv_accuracy.append(acc)
    
print('평균 정확도 : ', np.mean(cv_accuracy))

학습 데이터 크기 : 100, 검증 데이터 크기 : 50, 정확도 : 0.980000
학습 데이터 크기 : 100, 검증 데이터 크기 : 50, 정확도 : 0.940000
학습 데이터 크기 : 100, 검증 데이터 크기 : 50, 정확도 : 0.980000
평균 정확도 :  0.9666666666666667


# 교차검증을 이용해 간단하게

In [44]:
iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

features = iris.data
label = iris.target

scores = cross_val_score(dt_clf, features, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도 : ', scores)
print('평균 검증 정확도 : ', np.round(np.mean(scores), 4))

교차 검증별 정확도 :  [0.98 0.94 0.98]
평균 검증 정확도 :  0.9667


# 교차 검증과 최적의 하이퍼파라미터 튜닝을 한번에

In [48]:
iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=121, test_size=0.2)

dt_clf = DecisionTreeClassifier()

parameters = {'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]}
#하이퍼파라미터는 딕셔너리 형식으로 지정
#key : 결정트리의 하이퍼파라미터
#value : 하이퍼파라미터의 값

In [51]:
grid_tree = GridSearchCV(dt_clf, param_grid=parameter, cv=3, refit=True, return_train_score=True)

grid_tree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_tree.cv_results_)

In [52]:
#GridSearchCV 겨로가 세트로 딕셔너리 형태은 cv_results_를 DataFrame으로 변환 후 일부 파라미터 확인
scores_df[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1


In [54]:
#최고 성능을 가지는 파라미터 조합 및 예측 성능 1위값 출력
print('최적 파라미터 : ', grid_tree.best_params_)
print('최고 정확도 : ', grid_tree.best_score_)

최적 파라미터 :  {'max_depth': 3, 'min_samples_split': 2}
최고 정확도 :  0.975


In [57]:
#GridSeacrchCV 객체의 생성 파라미터로 refit=True로 설정된 경우(디폴트)
#GridSeacrchCV가 최적 성능을 나타내는 하이퍼 파라미터로 Estimator를 학습하고 best_estimaotr_로 저장(GridSeacrchCV의 refit으로 이미 학습이 된 estimator)
best_dt = grid_tree.best_estimator_

#best_estimator_는 이미 최적 학습이 됐으므로 별도 학습 필요 없이 바로 예측 가능
pred = best_dt.predict(X_test)
accuracy_score(y_test, pred)

0.9666666666666667